In [4]:
import json
import requests
import pandas as pd

fitcloud_url = "https://aws-dev.fitcloud.co.kr"
corpId = "KDjAqAG0TnEAAAFK5eqDUL0A"

# account 별 월 별 사용량 (saving plan 포함)
# type: usage - ApplySavingsPlanCompute
def corp_month(
    start_month: str, 
    end_month: str,
    token: str,
    groupBy="account",
    ):
  api_url = fitcloud_url + "/service/trend/corp/month"
  cookies = {
    "JSESSIONID": token,
  }

  data = {
      "from": start_month,
      "to": end_month,
      "groupBy": groupBy,
  }

  resp = requests.post(api_url, json=data, cookies=cookies)

  if resp.status_code == 200:
    # 일반 월 사용량에서 SavingPlan 가격을 제함
    return pd.DataFrame(resp.json())

  else:
    print("error")
#------------------------------------------------------------------------------------
# 월 입력값이  from: '201901', to: '202210'형태일 경우
# 시작 월부터 종료 월까지 리스트로 출력
from datetime import datetime, timedelta

def month_range(start_month, end_month):
    # Create datetime objects for start and end dates
    start_date = datetime.strptime(start_month, "%Y%m")
    end_date = datetime.strptime(end_month, "%Y%m")

    # Initialize list to store months
    months_list = []

    # Iterate over months and add them to the list
    current_month = start_date
    while current_month <= end_date:
        months_list.append(current_month.strftime("%Y%m"))
        current_month = (current_month + timedelta(days=32)).replace(day=1)

    return months_list
#-------------------------------------------------------------------------------------
# account 일자별 사용량을 반환
def ondemand_account_day(
    accountId: str, 
    day_from: str, 
    day_to: str, 
    token: str) -> float:
  api_url = fitcloud_url + "/ondemand/account/day"
  cookies = {
    "JSESSIONID": token,
  }

  data = {
      "from": day_from,
      "to": day_to,
      "accountId": accountId,
  }
  resp = requests.post(api_url, json=data, cookies=cookies)

  if resp.status_code == 200:
    # JSON 형식으로 응답을 파싱 후 usageFee 합계를 구하기 위해 dataframe 의 변환
    df = pd.DataFrame(resp.json())
    usage_sum = round( df['usage_fee'].astype("Float32").sum(), 2)
    return usage_sum

  else:
    print("error")

def corp_month_internal(start_month: str, end_month: str, accountId: str, token: str):
  """calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars.
  """
  json_data = corp_month(start_month, end_month, token)
  df = pd.DataFrame(json_data)
  # accountId = accountId
  # account에 관련된 데이터 추출
  df = df.query("accountId==@accountId")
  # 기간 내 월 리스트 추출
  month_list = month_range(start_month, end_month)
  # 월 column의 data type을 numeric으로 변환
  df_acc = df.copy()
  df_acc[month_list] = df_acc[month_list].apply(pd.to_numeric)
  # 내부 사용자용 filter: 합산에 포함시킬 항목
  internal_filter = ['Usage','ApplySavingsPlanCompute', 'ApplyRI' ]
  # internal_filter = ['Usage','ApplySavingsPlanCompute']
  df_int = df_acc.query("type in @internal_filter")
  sum = df_int[month_list].sum().sum()
  return sum

   

In [1]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun
)

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, tool


class FitUsageInput(BaseModel):
    start_month: str = Field(description="the starting month in period")
    end_month: str = Field(description="the end month in period")
    accountId: str = Field(description="account id")
    token: str = Field(description="JSESSIONID")

class FitUsageTool(BaseTool):
    name = "get_usage"
    description = "calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars."
    args_schema: type[BaseModel] = FitUsageInput

    def _run(self, start_month, end_month, accountId, token, int, run_manager: Optional[CallbackManagerForToolRun] = None) -> float:
        json_data = corp_month(start_month, end_month, token)
        df = pd.DataFrame(json_data)
        # accountId = accountId
        # account에 관련된 데이터 추출
        df = df.query("accountId==@accountId")
        # 기간 내 월 리스트 추출
        month_list = month_range(start_month, end_month)
        # 월 column의 data type을 numeric으로 변환
        df_acc = df.copy()
        df_acc[month_list] = df_acc[month_list].apply(pd.to_numeric)
        # 내부 사용자용 filter: 합산에 포함시킬 항목
        internal_filter = ['Usage','ApplySavingsPlanCompute', 'ApplyRI' ]
        # internal_filter = ['Usage','ApplySavingsPlanCompute']
        df_int = df_acc.query("type in @internal_filter")
        sum = df_int[month_list].sum().sum()
        return sum
    
    async def _arun(self, start_month, end_month, accountId, token, int, run_manager: Optional[CallbackManagerForToolRun] = None) -> float:
        """Use the tool asynchronously."""
        raise NotImplementedError("Calculator does not support async")


In [2]:
fit_usage = FitUsageTool()
print(fit_usage.name)
print(fit_usage.description)
print(fit_usage.args)

get_usage
calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars.
{'start_month': {'title': 'Start Month', 'description': 'the starting month in period', 'type': 'string'}, 'end_month': {'title': 'End Month', 'description': 'the end month in period', 'type': 'string'}, 'accountId': {'title': 'Accountid', 'description': 'account id', 'type': 'string'}, 'token': {'title': 'Token', 'description': 'JSESSIONID', 'type': 'string'}}


In [5]:
# StructuredTool dataclass
# https://blog.langchain.dev/structured-tools/
from langchain.tools.base import StructuredTool

usage = StructuredTool.from_function(
    func=corp_month_internal,
    name="get_usage",
    description="calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars"
)

In [ ]:
start_month = "202309"
end_month = "202309"
accountId = "532805286864"
token = ""
corp_month_internal(start_month=start_month, end_month=end_month, accountId=accountId, token=token)


In [4]:
import boto3
from langchain.llms.bedrock import Bedrock

# Create bedrock runtime client
session = boto3.session.Session(region_name='us-east-1')
br_client = session.client("bedrock-runtime")

llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=br_client,
    model_kwargs={
        "temperature": 0
    }
)



In [5]:
llm.invoke("who is steve jobs?")

' Steve Jobs (1955-2011) was an American entrepreneur and business magnate. He was the co-founder, chairman, and CEO of Apple Inc. Some key facts about Steve Jobs:\n\n- Co-founded Apple with Steve Wozniak in 1976 and led the company for over 3 decades until his death in 2011. Credited with revolutionizing personal computers, digital music players, smartphones, and tablet computers.\n\n- Pioneered innovations like the iMac, iPod, iPhone, and iPad. Helped popularize the computer mouse, email, and laser printers.\n\n- Co-founded animation studio Pixar and served as its CEO. Pixar movies like Toy Story pioneered computer animation techniques. \n\n- Known for his dynamic leadership style, perfectionism, and unconventional business approach. Often wore jeans and a black turtleneck.\n\n- Dropped out of college and traveled through India seeking enlightenment before co-founding Apple. Later returned to run Apple after being ousted in 1985.\n\n- Died in 2011 at age 56 after a lengthy battle wit